In [8]:
pip install tldextract

In [9]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
import re
import tldextract
from urllib.parse import urlparse
import logging
logging.getLogger('matplotlib.font_manager').setLevel(level=logging.CRITICAL)

from nltk.tokenize import word_tokenize
from wordcloud import WordCloud,STOPWORDS
from nltk.tokenize import RegexpTokenizer # regexp tokenizers use to split words from text
from nltk.stem.snowball import SnowballStemmer # stemmes words
import time # calculate time

#Garbage
import gc

#Warnings
import warnings
warnings.filterwarnings("ignore")

#SKLearn Libraries

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.ensemble import StackingClassifier
from scipy.sparse import hstack
import scipy.sparse

#WordCloud Generator
from wordcloud import WordCloud,STOPWORDS

#Plotting Library
import matplotlib.pyplot as plt
import seaborn as sns

# Telechargement des données

In [10]:
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilename

def upload_and_read_csv():
    """
    This function opens a file dialog to select a CSV file, reads the file into a pandas DataFrame,
    and returns the DataFrame.

    Returns:
        pd.DataFrame: DataFrame containing the data from the CSV file.
    """
    # Hide the root window
    Tk().withdraw()

    # Open file dialog
    filename = askopenfilename(title="Select CSV File", filetypes=[("CSV files", "*.csv")])

    # Check if a file was selected
    if not filename:
        print("No file selected.")
        return None

    # Read the CSV file into a DataFrame
    try:
        df = pd.read_csv(filename)
        print(f"File '{filename}' successfully loaded.")
        return df
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

In [6]:
# Example usage
data = upload_and_read_csv()
if data is not None:
    print(data.head())

Saving dataset.csv to dataset (1).csv
File 'dataset (1).csv' successfully loaded.
                                              domain  label
0  http://sprint.com/business/solutions/mobility_...    0.0
1              hesperia.gsfc.nasa.gov/~schmahl/nnls/    0.0
2  http://web.de/magazine/digital/sicher-im-netz/...    0.0
3  www.wired.com/science/discoveries/news/1999/08...    0.0
4   thomas.arnatt.com/wp-includes/pono/AOL_Logio.htm    1.0


# Pretraitement des données

In [13]:
# Fonction de nettoyage des données
def clean_data(df):
    # Supprimer les doublons Suppression des doublons dans un ensemble de données.
    # Les doublons sont un cas extrême d'échantillonnage non aléatoire et ils peuvent fausser le modèle ajusté.
    # Les inclure conduira essentiellement à un surajustement du modèle pour ce sous-ensemble de points.
    df = df.drop_duplicates()

    return df.dropna() # Suppression des lignes avec des valeurs manquantes

# Fonction de tokenisation, découper les URL en tokens (segments) pour extraire des caractéristiques textuelles
def tokenize_url(df):
    df['domain'] = df['domain'].str.lower()
    tokenizer = RegexpTokenizer(r'[A-Za-z]+')
    df['text_tokenized'] = df.domain.map(lambda t: tokenizer.tokenize(t)) # doing with all rows
    return df

def sentence_transfo(df):
  #Snowball est un petit langage de traitement des chaînes de caractères, qui donne des mots racines.
  #Dans ce cas, j'ai choisi l'anglais comme langue car les urls sont en anglais.
  stemmer = SnowballStemmer("english")

  df['text_stemmed'] = df['text_tokenized'].map(lambda l: [stemmer.stem(word) for word in l])
  df['text_sent'] = df['text_stemmed'].map(lambda l: ' '.join(l))

  return df

# Extraction des composants de l'URL
def extract_url_features(df):
    # Ajouter le schéma (http) si l'URL n'en a pas
    df['domain'] = df['domain'].apply(lambda x: "http://" + x if not x.startswith(("http://", "https://")) else x)

    # Extraire les composants de l'URL
    df['parsed_url'] = df['domain'].apply(lambda x: urlparse(x))
    df['domain_main'] = df['parsed_url'].apply(lambda x: x.netloc)
    df['path'] = df['parsed_url'].apply(lambda x: x.path)
    df['query'] = df['parsed_url'].apply(lambda x: x.query)
    df['query'] = df['query'].replace('','None')
    df["tld"] = df.domain_main.apply(lambda nl: tldextract.extract(nl).suffix)
    df['tld'] = df['tld'].replace('','None')

    """
    Les sections suivantes concernent certaines ponctuations dans l'URL qui peuvent indiquer d'une manière ou d'une autre qu'une
    URL est malveillante. Mon raisonnement est le suivant : les domaines typosquattés (qui sont presque toujours malveillants) peuvent
    contenir cette ponctuation pour ressembler à un domaine légitime. Il peut également y avoir plus de ces ponctuations dans le chemin
    de l'URL pour une URL légitime, car les blogs utilisent souvent des underscores dans une URL.
    """

    # Longueur de l'URL
    df['url_length'] = df['domain'].apply(lambda x: len(x))

    # Nombre de sous-domaines
    df['num_subdomains'] = df['domain_main'].apply(lambda x: len(x.split('.')) - 2)

    # Nombre de paramètres dans la query
    df['num_params'] = df['query'].apply(lambda x: len(x.split('&')) if x else 0)

    df['path_hyphens'] = df.path.str.count('-')
    df['path_underscores'] = df.path.str.count('_')
    df['slashes'] = df.path.str.count('/')

    """
    Les points dans le chemin d'accès peuvent indiquer une tentative de tromper l'utilisateur en lui faisant croire que le domaine est
    légitime. Par exemple, example.com/bankofamerica.com peut induire en erreur. De plus, les points peuvent signaler la présence de
    fichiers malveillants, comme malware.exe, dans l'URL.
    """
    df['full_stops'] = df.path.str.count('.')

    # Fréquence des caractères spéciaux
    df['num_special_chars'] = df['domain'].apply(lambda x: len(re.findall(r'[\W_]', x)))

    return df

def preprocess(df):
  df1 = clean_data(df)
  df2 = tokenize_url(df1)
  df3 = sentence_transfo(df2)
  df4 = extract_url_features(df3)

  # Sauvegarder le DataFrame prétraité
  df4.to_csv('preprocessed_data.csv', index=False)
  return df4

preprocessed_data = preprocess(data)

preprocessed_data.head()

,domain,label,text_tokenized,text_stemmed,text_sent,parsed_url,domain_main,path,query,tld,url_length,num_subdomains,num_params,path_hyphens,path_underscores,slashes,full_stops,num_special_chars
0,http://sprint.com/business/solutions/mobility_...,0.0,"[http, sprint, com, business, solutions, mobil...","[http, sprint, com, busi, solut, mobil, secur,...",http sprint com busi solut mobil secur and man...,"(http, sprint.com, /business/solutions/mobilit...",sprint.com,/business/solutions/mobility_security_and_mana...,intnav=atg:he:mobsecmgmt,com,105,0,1,0,3,4,63,16
1,http://hesperia.gsfc.nasa.gov/~schmahl/nnls/,0.0,"[hesperia, gsfc, nasa, gov, schmahl, nnls]","[hesperia, gsfc, nasa, gov, schmahl, nnls]",hesperia gsfc nasa gov schmahl nnls,"(http, hesperia.gsfc.nasa.gov, /~schmahl/nnls/...",hesperia.gsfc.nasa.gov,/~schmahl/nnls/,None,gov,44,2,1,0,0,3,15,10
2,http://web.de/magazine/digital/sicher-im-netz/...,0.0,"[http, web, de, magazine, digital, sicher, im,...","[http, web, de, magazin, digit, sicher, im, ne...",http web de magazin digit sicher im netz siche...,"(http, web.de, /magazine/digital/sicher-im-net...",web.de,/magazine/digital/sicher-im-netz/sicherheitshi...,None,de,131,0,1,10,0,4,118,18
3,http://www.wired.com/science/discoveries/news/...,0.0,"[www, wired, com, science, discoveries, news]","[www, wire, com, scienc, discoveri, news]",www wire com scienc discoveri news,"(http, www.wired.com, /science/discoveries/new...",www.wired.com,/science/discoveries/news/1999/08/21495,None,com,59,1,1,0,0,6,39,11
4,http://thomas.arnatt.com/wp-includes/pono/aol_...,1.0,"[thomas, arnatt, com, wp, includes, pono, aol,...","[thoma, arnatt, com, wp, includ, pono, aol, lo...",thoma arnatt com wp includ pono aol logio htm,"(http, thomas.arnatt.com, /wp-includes/pono/ao...",thomas.arnatt.com,/wp-includes/pono/aol_logio.htm,None,com,55,1,1,1,1,3,31,11
